In [76]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash,dash_table,Input,Output,html,dcc
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template as lft
import dash_bootstrap_components as dbc

In [12]:
Customer=pd.read_csv(r"C:\Users\akhia\Documents\Python dataanalysis\Resume project\dim_customers.csv")
Payment=pd.read_csv(r"C:\Users\akhia\Documents\Python dataanalysis\Resume project\fact_spends.csv")

In [ ]:
pd.merge(Customer,avg_spend,on='customer_id',how='left')

In [37]:
avg_spend=Payment.groupby('customer_id',as_index=False).agg({'spend':'sum'})

In [38]:
Avg_df=pd.merge(Customer,avg_spend,on='customer_id',how='left')

In [201]:
Full_df=pd.merge(Payment,Avg_df,on='customer_id',how='left').drop(columns='Average Spend')

In [52]:
Avg_df['Avg income utilisation']=Avg_df['Average Spend']/Avg_df['avg_income']

In [42]:
Avg_df.rename(columns={'spend':'Average Spend'},inplace=True)

In [334]:
Full_df['month'].unique()

array(['July', 'October', 'May', 'September', 'June', 'August'],
      dtype=object)

In [338]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app=Dash(__name__,external_stylesheets=[dbc.themes.SLATE,dbc_css])
lft('SLATE')
app.layout=html.Div([
    dcc.Tabs(
    className='custom-tabs-container',
    children=[
        dcc.Tab(id='tab 0',label='Data Set',className='dbc',  #Tab 1
               children=[
                   html.H1(" Data Set in Different City",style={
                       'color':"#fa0a6a",
                       'text-align':'center',
                       'fontFamily':'Arial'
                       
                   }),
                   dbc.Row(dcc.RadioItems(id='city',           #Radioitems for different city
                                options=Full_df['city'].unique(),
                                value='Bengaluru',className='dbc',inline=True,
                                )),
                   html.Br(),
                   dbc.Row(html.Div(id='Table') )                
                   
               ]
               ),
        dcc.Tab( id='tab 1',label='Avg income utilisation ',className='dbc',   #Tab 2
                children=[  
                            dbc.Row(
                                
                                html.H1("Average income utilisation of different catagories",
                                       style={  'color':"#fa0a6a",
                                                'text-align':'center',
                                                'fontFamily':'Arial'}
                                       )),
                        
                        html.Br(),
                            dbc.Row(dcc.RadioItems(id='check',    #Radioitems for different category
                                       options=
                                                   [    {'label': '  Age Group ', 'value': 'age_group'},
                                                        {'label': '  City ', 'value': 'city'},
                                                        {'label': '  Occupation ', 'value': 'occupation'},
                                                        {'label': '  Gender ', 'value': 'gender'},
                                                        {'label': '  Marital status ', 'value': 'marital status'}],
                                       value='occupation',inline=True,
                                                  className='dbc')),
                        html.Br(),
                            dbc.Row(
                                  children=
                                              [dbc.Col(dcc.Graph(id='fig')),       #bar graphs in Tab 2
                                               html.Br(),
                                                dbc.Col(dcc.Graph(id='fig1-2')) ] 
                          
                                  ) 
                                
                                
                                ]),
        dcc.Tab(id='tab 2',label='Spending Insights and Payment Methods',    #tab 3
                children=[dbc.Row(html.H1("Payment Method analysis",
                                         style={  'color':"#fa0a6a",
                                                'text-align':'center',
                                                'fontFamily':'Arial'})),
                         html.Br(),
                         dbc.Row(
                                 children=[
                                     html.H2('Categories'),
                                     dbc.Col(dcc.RadioItems(id='check2',          #Radioitems for different category
                                               options=Full_df['category'].unique(),
                                               value=['Food'],className='dbc'),width=3),
                                     html.Br(),
                                     dbc.Col(dcc.Graph(id='fig2'),width=7),      #Line Graph
                                     dbc.Col(width=2)
                                         ]
                                 ),
                          html.Br(),
                          dbc.Row(
                                 children=[
                                     html.H2('Month'), 
                                     dbc.Col(dcc.RadioItems(id='check2-1',      #Radioitems for different month
                                               options=Full_df['month'].unique(),
                                               value=['May'],className='dbc')
                                             ,width=2),
                                     html.Br(),
                                     dbc.Col(dcc.Graph(id='fig2-1'),width=5),     #pie charts 
                                     dbc.Col(dcc.Graph(id='fig2-2'),width=5)
                                         ]
                                 )
                         
                         
                         ])
        
    ], colors={                             #Tab Header and style
        "border": "grey",
        "primary": "gold",
        "background": "#0f1133",
        
    },style={'fontSize':22,
             'fontWeight': 'bold'}
    )    
])

@app.callback(Output('Table','children'),             #Tab 1 Callback
              Input('city','value'))

def table_op(c):
    df=Full_df.query("city == @c and payment_type =='Credit Card'")         #dash table
    table=dash_table.DataTable(
        columns=[{"name":i,'id':i}for i in df.columns],
        data=df.to_dict('records'),
        filter_action='native',
        sort_action='native',
        export_format='csv',
        page_size=20,
        style_header={
            'color':'Black',
            'backgroundColor': 'lightblue',
            'fontWeight': 'bold'
        },
        style_cell_conditional=[
                {
                    'if': {'column_id': c},
                    'textAlign': 'left'
                } for c in ['customer_id','month']
            ],
        style_data={
            'color':'Black',
            'backgroundColor': 'lightgrey',
            
        }
#    style_as_list_view=True,
      
    )
    return table

@app.callback(Output('fig','figure'),                 #Tab 2 Callback
              Output('fig1-2','figure'),
             Input('check','value'))


def plot(v):
    
    df=(Avg_df.groupby( v,as_index=False)
               .agg({'Avg income utilisation':'mean'}
                ).sort_values(['Avg income utilisation'],ascending=False))
    
    fig=px.bar(df,                                                        #bar graph 1
                  x=v,y='Avg income utilisation', 
                   title=F"Mean Avgerage Income Spend of Different {v}"
      ).update_layout(title_x=.5,title_y=.98)
    
    fig2=px.bar((Avg_df.groupby( v,as_index=False)                         #bar graph 2
               .agg({'avg_income':'mean','Average Spend':'mean'}
                ).sort_values(['Average Spend','avg_income'],ascending=False)),
                  x=v,y=[ 'Average Spend','avg_income'], barmode='group',
                   title=F"Income and Spend - {v}"
      ).update_layout(title_x=.5,title_y=.98)
    return fig, fig2

@app.callback(Output('fig2','figure'),                 #Tab 3 Callback
              Output('fig2-1','figure'),
              Output('fig2-2','figure'),
             Input('check2','value'),
             Input('check2-1','value'))


def plot(t,m):
    
    
    
    fig=px.line((Full_df.groupby(['category','month','payment_type'],as_index=False).agg({'spend':'sum'})     #line graph 
                .query("category == @t")),
         x='month',y='spend',
        color="payment_type",
        title="Different Payment Methods"
       ).update_layout(title_x=.5,title_y=.95)
    
    fig2=px.pie(((Full_df.groupby(['month','payment_type'],as_index=False).agg({'spend':'sum'})  #pie graph 1
                .query("month == @m"))),
       values='spend',names='payment_type'
               ).update_layout(
        title=f'Different Payment Methods {m} month',
        title_x=.5,title_y=.95)
    fig3=px.pie(((Full_df.groupby(['month','category'],as_index=False).agg({'spend':'sum'})  #pie graph 2
                .query("month == @m"))),
       values='spend',names='category').update_layout(
        title=f'Different Categories {m} month',
        title_x=.5,title_y=.95)
    
    return fig  ,fig2,fig3
    
if __name__=='__main__':
    app.run_server(
        jupyter_mode='tab',
        port=8336)

Dash app running on http://127.0.0.1:8336/


<IPython.core.display.Javascript object>